In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import networkx as nx
import pickle
import os

%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
data_dir = r"E:\tencent files\chrome Download\Research\LCA\LCA Coal_fired power plant phaseout\data"
distance_dir_year = r"coal_transport2010"
distance_dir_matrix = r"Distance_matrix"
distance_dir = os.path.join(data_dir, distance_dir_year, distance_dir_matrix)

<IPython.core.display.Javascript object>

In [3]:
list_file = os.listdir(distance_dir)

<IPython.core.display.Javascript object>

In [4]:
len(list_file)

262

<IPython.core.display.Javascript object>

In [5]:
%%time
Distance_matrix = pd.DataFrame()
for i in range(len(list_file)):
    with open(os.path.join(distance_dir, list_file[i]), "rb") as file:
        dis_df = pickle.load(file)['path_length']
        Distance_matrix[list_file[i][:6]]=dis_df

Wall time: 3min 25s


<IPython.core.display.Javascript object>

In [6]:
Distance_matrix.to_excel(
    os.path.join(data_dir, distance_dir_year) + "\path_length.xlsx"
)

<IPython.core.display.Javascript object>

In [58]:
%%time
path_matrix = pd.DataFrame()
for i in range(len(list_file)):
   with open(os.path.join(distance_dir, list_file[i]), "rb") as file:
        dis_df = pickle.load(file)['geometry']
        path_matrix[list_file[i][:6]]=dis_df

Wall time: 11min 37s


<IPython.core.display.Javascript object>

In [ ]:
with open(os.path.join(data_dir, distance_dir_year,"path.pickle") ,'wb') as file:
    pickle.dump(path_matrix,file)

In [8]:
from shapely.geometry import LineString, Point, Polygon

<IPython.core.display.Javascript object>

## 公路运距

In [9]:
road_length_power = (
    pd.read_excel(os.path.join(data_dir, distance_dir_year, "node_power_pair.xlsx"))
    .groupby("node_id")["length"]
    .mean()
)
road_length_power

node_id
119318       997.538437
119320       614.591250
119322      1890.030034
119324      6294.310051
119326      7648.634567
              ...      
119672    111471.697265
119673     10244.661299
119674       388.698779
119676       591.651267
119677     22956.421760
Name: length, Length: 262, dtype: float64

<IPython.core.display.Javascript object>

In [10]:
road_length_mine = (
    pd.read_excel(os.path.join(data_dir, distance_dir_year, "node_mine_pair.xlsx"))
    .groupby("node_id")["length"]
    .mean()
)
road_length_mine

node_id
116330     7459.796010
116331    14812.624847
116332    12117.941804
116333     1112.508772
116335     2164.514319
              ...     
119311    40665.418460
119312    27327.391198
119313     7738.712582
119315     6496.603399
119317    96355.265326
Name: length, Length: 2190, dtype: float64

<IPython.core.display.Javascript object>

In [11]:
road_path = pd.DataFrame(
    np.array(road_length_power).reshape(1, len(road_length_power))
    + np.array(road_length_mine).reshape(len(road_length_mine), 1),
    index=road_length_mine.index,
    columns=road_length_power.index,
)

<IPython.core.display.Javascript object>

In [15]:
road_path.head().iloc[:, :12]

node_id,119318,119320,119322,119324,119326,119327,119328,119329,119331,119332,119333,119334
node_id,,,,,,,,,,,,
116330,8457.334447,8074.387260,9349.826044,13754.106061,15108.430577,100743.502765,11506.265979,10305.754517,17468.935713,23441.301868,88062.838902,52255.504657
116331,15810.163284,15427.216097,16702.654881,21106.934898,22461.259414,108096.331602,18859.094816,17658.583354,24821.764550,30794.130705,95415.667739,59608.333494
116332,13115.480241,12732.533053,14007.971838,18412.251855,19766.576371,105401.648558,16164.411772,14963.900311,22127.081507,28099.447662,92720.984695,56913.650450
116333,2110.047209,1727.100022,3002.538806,7406.818823,8761.143339,94396.215527,5158.978741,3958.467280,11121.648476,17094.014631,81715.551664,45908.217419
116335,3162.052756,2779.105569,4054.544353,8458.824370,9813.148886,95448.221074,6210.984288,5010.472826,12173.654022,18146.020177,82767.557211,46960.222966


<IPython.core.display.Javascript object>

## 铁路运距

In [16]:
distance_matrix = pd.read_excel(
    os.path.join(data_dir, distance_dir_year, "path_length.xlsx"), index_col=0
)

<IPython.core.display.Javascript object>

In [17]:
rail_path = pd.DataFrame(
    np.array(distance_matrix) - np.array(road_path),
    index=road_length_mine.index,
    columns=road_length_power.index,
)

<IPython.core.display.Javascript object>

In [18]:
road_price = 0.08417
rail_price = 0.01544
road_cost = road_path * road_price
rail_cost = rail_path * rail_price

<IPython.core.display.Javascript object>

In [19]:
transport_cost_matrix = road_cost + rail_cost

<IPython.core.display.Javascript object>

In [20]:
road_path.to_excel(os.path.join(data_dir, distance_dir_year, "road_path_length.xlsx"))
rail_path.to_excel(os.path.join(data_dir, distance_dir_year, "rail_path_length.xlsx"))

<IPython.core.display.Javascript object>

## 煤炭交流矩阵

In [21]:
coal_transport_mat = pd.read_excel(
    os.path.join(data_dir, distance_dir_year, "coal_transport_mat2010.xlsx")
)

<IPython.core.display.Javascript object>

In [22]:
coal_transport_mat_fillna = coal_transport_mat.fillna(value=0).iloc[1:, :]
coal_transport_mat_fillna.head()

,到达\r\n发送,北京,天津,河北,山西,内蒙古,辽宁,吉林,黑龙江,上海,...,四川,贵州,云南,西藏,陕西,甘肃,青海,宁夏,新疆,合计
1,北京,2.0,173.0,160.0,0.0,0.0,52.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,436.0
2,天津,0.0,2582.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2599.0
3,河北,370.0,2066.0,3786.0,19.0,24.0,197.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7477.0
4,山西,833.0,2297.0,28152.0,3347.0,110.0,1271.0,28.0,19.0,15.0,...,0.0,0.0,0.0,0.0,33.0,1.0,0.0,0.0,94.0,52340.0
5,内蒙古,285.0,549.0,10023.0,0.0,5560.0,6486.0,2786.0,2426.0,0.0,...,0.0,0.0,0.0,0.0,0.0,79.0,0.0,108.0,0.0,28349.0


<IPython.core.display.Javascript object>

In [23]:
from translate import Translator

translator = Translator(from_lang="zh", to_lang="en")

<IPython.core.display.Javascript object>

In [24]:
coal_transport_mat_fillna["start_province"] = coal_transport_mat_fillna[
    "到达\r\n发送"
].apply(lambda x: translator.translate(x))

<IPython.core.display.Javascript object>

In [25]:
coal_transport_mat_SD = coal_transport_mat_fillna.sort_values("start_province").loc[
    :, ["start_province", "山东"]
]
coal_transport_mat_SD.loc[7, "start_province"] = "Jilin"

<IPython.core.display.Javascript object>

In [26]:
coal_transport_mat_SD_dict = {
    i: j
    for i, j in zip(
        coal_transport_mat_SD["start_province"], coal_transport_mat_SD["山东"]
    )
}

<IPython.core.display.Javascript object>

In [27]:
coal_transport_mat_SD_dict

{'Anhui': 260.0,
 'Beijing': 43.0,
 'Chongqing': 0.0,
 'Fujian': 0.0,
 'Gansu': 66.0,
 'Guangdong': 0.0,
 'Guangxi': 0.0,
 'Guizhou': 0.0,
 'Hainan': 0.0,
 'Hebei': 762.0,
 'Heilongjiang': 0.0,
 'Henan': 334.0,
 'Hubei': 0.0,
 'Hunan': 0.0,
 'Inner Mongolia': 48.0,
 'Jiangsu': 78.0,
 'Jiangxi': 0.0,
 'Jilin': 0.0,
 'Liaoning': 0.0,
 'Ningxia': 43.0,
 'Qinghai': 9.0,
 'Shaanxi': 791.0,
 'Shandong': 2332.0,
 'Shanghai': 0.0,
 'Shanxi': 8203.0,
 'Sichuan': 0.0,
 'Tianjin': 0.0,
 'Tibet': 0.0,
 'Xinjiang': 0.0,
 'Yunnan': 0.0,
 'Zhejiang': 0.0}

<IPython.core.display.Javascript object>

## 计算煤矿到电厂的煤炭运量

In [28]:
power_data = pd.read_excel(
    os.path.join(data_dir, distance_dir_year, "node_power_pair.xlsx")
)
power_data.head(2)

,Unnamed: 0,总序号,省序号,厂序号,省份,电厂,区划代码,地级市/自治州,发电类别,年末发电设备容量\n（千瓦）,...,发电厂用电率\n（%）,发电标准煤耗\r\n（克/千瓦时）,供电标准煤耗率\n（克/千瓦时）,发电耗用原煤量\n（吨）,longitude,latitude,geometry,intersection_point,length,node_id
0,0,2537,15,3,山东省,菏泽发电厂#3對机,371700,菏泽市,NaN,600000.0,...,5.2,317.0,334.0,1506056.0,115.545693,35.253672,POINT (4274728.137388794 4757028.049643864),POINT (4275429.892443932 4756319.090705633),997.538437,119318
1,1,2548,15,14,山东省,菏泽发电厂1-2#,371700,菏泽市,NaN,250000.0,...,8.9,350.0,385.0,754032.0,115.545693,35.253672,POINT (4274728.137388794 4757028.049643864),POINT (4275429.892443932 4756319.090705633),997.538437,119318


<IPython.core.display.Javascript object>

In [29]:
mine_capacity2010 = "coal_prod2010_SD.xlsx"
mine_capacity2012 = "node_mine_pair_withCapacity2.xlsx"
mine_capacity2014 = "coal_prod2014_SD.xlsx"

<IPython.core.display.Javascript object>

In [30]:
mine_capacity = pd.read_excel(
    os.path.join(data_dir, distance_dir_year, mine_capacity2010)
)
mine_capacity.head()

,IDNum,Province,PrefMunCty,CntyDstCty,LatDD,LongDD,Rank,MineSize,MineName,Unnamed: 9,allocation,矿山容量（万吨）,10火力发电占比（全国平均）,Capacity,Unnamed: 14,Unnamed: 15,矿山每份销量（万吨）,Unnamed: 17,煤炭全国销量10/13
0,1.0,Anhui Province,Suzhou City,Dangshan County,34.331641,116.482994,Bituminous,Large,Dang shan,NaN,10,364.965131,0.382138,139.467148,NaN,安徽省,42.2944,NaN,0.862917
1,2.0,Anhui Province,Suzhou City,Xiao County,34.250348,116.596540,Bituminous,Small,Zhang datun,NaN,1,36.496513,0.382138,13.946715,NaN,NaN,NaN,NaN,NaN
2,3.0,Anhui Province,Suzhou City,Xiao County,34.130176,116.787770,Bituminous,Medium,Tang miao,NaN,4,145.986052,0.382138,55.786859,NaN,NaN,NaN,NaN,NaN
3,4.0,Anhui Province,Huaibei City,NaN,34.023946,116.831329,Bituminous,Medium,Dai he,NaN,4,145.986052,0.382138,55.786859,NaN,NaN,NaN,NaN,NaN
4,5.0,Anhui Province,Huaibei City,NaN,34.061551,116.887518,Bituminous,Medium,Shuo li,NaN,4,145.986052,0.382138,55.786859,NaN,NaN,NaN,NaN,NaN


<IPython.core.display.Javascript object>

In [31]:
mine_data = pd.read_excel(
    os.path.join(data_dir, distance_dir_year, "node_mine_pair.xlsx")
)
mine_data.head(3)

,Unnamed: 0,IDNum,Province,PrefMunCty,CntyDstCty,LatDD,LongDD,Rank,MineSize,MineName,geometry,index_right,pop_est,continent,name,iso_a3,gdp_md_est,intersection_point,length,node_id
0,0,1.0,Anhui Province,Suzhou City,Dangshan County,34.331641,116.482994,Bituminous,Large,Dang shan,POINT (4420001.203680727 4693619.627322398),139,1379302771,Asia,China,CHN,21140000,POINT (4417515.434006381 4700653.08348814),7459.796010,116330
1,1,2.0,Anhui Province,Suzhou City,Xiao County,34.250348,116.596540,Bituminous,Small,Zhang datun,POINT (4436002.57311189 4689567.753009823),139,1379302771,Asia,China,CHN,21140000,POINT (4427241.652733594 4677623.711441736),14812.624847,116331
2,2,3.0,Anhui Province,Suzhou City,Xiao County,34.130176,116.787770,Bituminous,Medium,Tang miao,POINT (4462033.413644917 4684761.124856576),139,1379302771,Asia,China,CHN,21140000,POINT (4473556.785094056 4681011.935077398),12117.941804,116332


<IPython.core.display.Javascript object>

In [32]:
mine_withCapacity = pd.merge(mine_data, mine_capacity, how="outer")
mine_withCapacity_sorted = mine_withCapacity.sort_values("node_id")
len(mine_withCapacity)

2443

<IPython.core.display.Javascript object>

In [33]:
mine_withCapacity

,Unnamed: 0,IDNum,Province,PrefMunCty,CntyDstCty,LatDD,LongDD,Rank,MineSize,MineName,...,Unnamed: 9,allocation,矿山容量（万吨）,10火力发电占比（全国平均）,Capacity,Unnamed: 14,Unnamed: 15,矿山每份销量（万吨）,Unnamed: 17,煤炭全国销量10/13
0,0.0,1.0,Anhui Province,Suzhou City,Dangshan County,34.331641,116.482994,Bituminous,Large,Dang shan,...,NaN,10.0,364.965131,0.382138,139.467148,NaN,安徽省,42.2944,NaN,0.862917
1,1.0,2.0,Anhui Province,Suzhou City,Xiao County,34.250348,116.596540,Bituminous,Small,Zhang datun,...,NaN,1.0,36.496513,0.382138,13.946715,NaN,NaN,NaN,NaN,NaN
2,2.0,3.0,Anhui Province,Suzhou City,Xiao County,34.130176,116.787770,Bituminous,Medium,Tang miao,...,NaN,4.0,145.986052,0.382138,55.786859,NaN,NaN,NaN,NaN,NaN
3,3.0,4.0,Anhui Province,Huaibei City,NaN,34.023946,116.831329,Bituminous,Medium,Dai he,...,NaN,4.0,145.986052,0.382138,55.786859,NaN,NaN,NaN,NaN,NaN
4,4.0,5.0,Anhui Province,Huaibei City,NaN,34.061551,116.887518,Bituminous,Medium,Shuo li,...,NaN,4.0,145.986052,0.382138,55.786859,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2438,NaN,35.0,Xizang (Tibet) Autonomous Region,Chamdo Prefecture,Dingqing (Dengqen) County,32.293162,94.897254,Anthracite,Small,Zi jia pu,...,NaN,1.0,36.496513,0.382138,13.946715,NaN,NaN,NaN,NaN,NaN
2439,NaN,36.0,Xizang (Tibet) Autonomous Region,Ngari Prefecture,Ga'er (Gar) County,31.370120,80.889489,Bituminous,Small,NaN,...,NaN,1.0,36.496513,0.382138,13.946715,NaN,NaN,NaN,NaN,NaN
2440,NaN,54.0,Xizang (Tibet) Autonomous Region,Naqu Prefecture,Naqu County,31.385650,92.944476,Lignite,Small,Se xiong,...,NaN,1.0,36.496513,0.382138,13.946715,NaN,NaN,NaN,NaN,NaN
2441,NaN,59.0,Xizang (Tibet) Autonomous Region,Chamdo Prefecture,Leiwuqi (Riwoqe ) County,31.424566,96.754160,Bituminous,Small,Ren qing,...,NaN,1.0,36.496513,0.382138,13.946715,NaN,NaN,NaN,NaN,NaN


<IPython.core.display.Javascript object>

In [34]:
mine_withCapacity_sorted_sumNode = mine_withCapacity.groupby(
    ["Province", "node_id"]
).sum()

<IPython.core.display.Javascript object>

In [35]:
mine_withCapacity_sorted_sumNode.loc["Shandong Province", "Capacity"].sum()

5160.284474966648

<IPython.core.display.Javascript object>

In [36]:
coal_trans_dict = {}
for province in coal_transport_mat_SD["start_province"]:
    node_set = []
    for (province_name, node) in mine_withCapacity_sorted_sumNode.index:
        if province in province_name:
            node_set.append(node)
    coal_trans_dict[province] = node_set

<IPython.core.display.Javascript object>

In [37]:
from pulp import *

<IPython.core.display.Javascript object>

In [38]:
prob = LpProblem("transportation", LpMinimize)

<IPython.core.display.Javascript object>

In [39]:
power_plant = transport_cost_matrix.columns
mine = transport_cost_matrix.index

<IPython.core.display.Javascript object>

In [40]:
power_data["发电耗用原煤量\n（吨）"].sum()

136114151.28239483

<IPython.core.display.Javascript object>

In [41]:
power_capacity = power_data.groupby("node_id")["发电耗用原煤量\n（吨）"].sum()

<IPython.core.display.Javascript object>

In [42]:
mine_capacity = (mine_withCapacity.groupby("node_id")["Capacity"].sum()) * 10000

<IPython.core.display.Javascript object>

In [43]:
# 供给与需求约束
demand = {name: demand for name, demand in zip(power_plant, power_capacity.values)}
supply = {name: supply for name, supply in zip(mine, mine_capacity)}

<IPython.core.display.Javascript object>

In [44]:
weight = transport_cost_matrix.to_dict()

<IPython.core.display.Javascript object>

In [45]:
route = [(p, m) for p in power_plant for m in mine]

<IPython.core.display.Javascript object>

In [46]:
route_var = LpVariable.dicts("route", (power_plant, mine), 0, None)

<IPython.core.display.Javascript object>

In [47]:
prob += lpSum([route_var[p][m] * weight[p][m] for (p, m) in route])

<IPython.core.display.Javascript object>

In [48]:
for p in power_plant:
    prob += lpSum([route_var[p][m]] for m in mine) >= demand[p]

for m in mine:
    prob += lpSum([route_var[p][m]] for p in power_plant) <= supply[m]

for coal_transport in coal_transport_mat_SD_dict.keys():
    if coal_transport != "Shandong":
        prob += (
            lpSum(
                [
                    route_var[p][m]
                    for p in power_plant
                    for m in mine
                    if m in coal_trans_dict[coal_transport]
                ]
            )
            <= coal_transport_mat_SD_dict[coal_transport] * 10000
        )
    else:
        prob += lpSum(
            [
                route_var[p][m]
                for p in power_plant
                for m in mine
                if m in coal_trans_dict[coal_transport]
            ]
        ) <= (coal_transport_mat_SD_dict[coal_transport] * 10000 / 0.27)

<IPython.core.display.Javascript object>

In [49]:
prob.solve()

1

<IPython.core.display.Javascript object>

In [50]:
LpStatus[prob.status]

'Optimal'

<IPython.core.display.Javascript object>

In [51]:
dic = {v.name: v.varValue for v in prob.variables()}

<IPython.core.display.Javascript object>

In [52]:
for i in route_var.keys():
    for j in route_var[i].keys():
        route_var[i][j] = dic[str(route_var[i][j])]

<IPython.core.display.Javascript object>

In [53]:
pd.DataFrame(route_var).to_excel("D:\desktop\coal_trasport2010.xlsx")

<IPython.core.display.Javascript object>